In [1]:
import pandas as pd
import numpy as np


import pickle

from utilities import PrecipFix, WindDirection

In [2]:
df = pickle.load( open( "seattle_data.pkl", "rb" ) )

In [ ]:
df.head()

In [ ]:
df[df.RECORDING_COUNT != 24]

In [ ]:
df.dropna(axis='columns',how='all', inplace=True)

In [ ]:
df.columns.tolist()

In [ ]:
df.drop(columns=['REM', 'REPORT_TYPE.1', 'SOURCE.1', 'WindEquipmentChangeDate', 'RECORDING_COUNT', 'REPORT_TYPE', 'SOURCE'], inplace=True)

In [ ]:
df.head()

**Drop columns with lots of missing values**

In [ ]:
df.drop(columns=df.columns[df.isna().sum() > 30000], inplace=True)

In [ ]:
df

**Second pass at columns with missing values**

In [ ]:
df.isna().sum()

In [ ]:
df.HourlyStationPressure.value_counts()

**Remove irrelevant or redundant columns**

In [ ]:
df.drop(columns=['STATION', 'HourlyAltimeterSetting', 'HourlyDewPointTemperature', 'HourlySeaLevelPressure', 'HourlyWetBulbTemperature'], inplace=True)

In [ ]:
df.isna().sum()

**Explore SkyConditions column**

In [ ]:
df.HourlySkyConditions = df.HourlySkyConditions.apply(lambda x: str(x)[:3])

In [ ]:
df.HourlySkyConditions.value_counts()

In [ ]:
df.loc[df.HourlySkyConditions == '*']

**just fill forward, commonly used in time series**

In [ ]:
df.fillna(method='ffill', inplace=True)

In [ ]:
df.isna().sum()

**Fix remaining sky conditions**

In [ ]:
df.loc[df.HourlySkyConditions == '*']

In [ ]:
df.HourlySkyConditions.iloc[40180:40187]

In [ ]:
df.HourlySkyConditions.iloc[40184] = 'OVC'

In [ ]:
df.loc[df.HourlySkyConditions == '*']

In [ ]:
for column in df.columns:
    print(df[column].value_counts())

**Fix Temperature**

In [ ]:
df.replace('*',np.NaN, inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df.fillna(method='ffill', inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df.HourlyDryBulbTemperature = df.HourlyDryBulbTemperature.apply(lambda x: PrecipFix(x))

In [ ]:
df.HourlyDryBulbTemperature.value_counts()

**Fix Precipitation**

In [ ]:
df.HourlyPrecipitation = df.HourlyPrecipitation.apply(lambda x: PrecipFix(x))

In [ ]:
df.HourlyPrecipitation.value_counts()

**Fix RH**

In [ ]:
df.HourlyRelativeHumidity = df.HourlyRelativeHumidity.apply(lambda x: PrecipFix(x))

In [ ]:
df.HourlyRelativeHumidity.value_counts()

**Fix sky conditions**

In [ ]:
df.HourlySkyConditions.value_counts()

In [ ]:
df.HourlySkyConditions.replace('VV:', 'VV', inplace=True)

In [ ]:
df.HourlySkyConditions.replace('nan', np.NaN, inplace=True)

In [ ]:
df.fillna(method='ffill', inplace=True)

**Fix station pressure**

In [ ]:
df.HourlyStationPressure.value_counts()

In [ ]:
df.HourlyStationPressure = df.HourlyStationPressure.apply(lambda x: PrecipFix(x))

**Fix Visibility**

In [ ]:
df.HourlyVisibility.replace('*', np.NaN, inplace=True)

In [ ]:
df.fillna(method='ffill', inplace=True)

In [ ]:
df.HourlyVisibility.value_counts()

In [6]:
df.HourlyVisibility = df.HourlyVisibility.apply(lambda x: PrecipFix(x))

In [7]:
for column in df.columns:
    print(df[column].value_counts())

2015-12-02 19:53:00    1
2015-08-23 16:53:00    1
2018-03-13 06:53:00    1
2019-10-06 13:53:00    1
2018-01-13 06:53:00    1
                      ..
2016-10-22 12:53:00    1
2019-08-21 18:53:00    1
2018-08-19 23:53:00    1
2017-08-07 19:53:00    1
2016-01-02 08:53:00    1
Name: DATE, Length: 47424, dtype: int64
45.0    1743
46.0    1694
48.0    1660
50.0    1633
47.0    1467
        ... 
97.0       3
96.0       3
17.0       1
16.0       1
15.0       1
Name: HourlyDryBulbTemperature, Length: 83, dtype: int64
0.00    43045
0.01     1584
0.02      804
0.03      519
0.04      341
0.05      258
0.06      208
0.07      135
0.08      128
0.09       88
0.10       77
0.11       55
0.12       44
0.13       31
0.14       27
0.15       14
0.18       13
0.16       10
0.19        8
0.17        8
0.20        6
0.22        4
0.24        4
0.25        3
0.27        3
0.26        2
0.23        2
1.04        2
0.21        1
Name: HourlyPrecipitation, dtype: int64
93.0    2342
89.0    2079
86.0    2027


**Fix Wind Direction**

In [3]:
df.HourlyWindDirection = df.HourlyWindDirection.apply(lambda x: WindDirection(x))

In [4]:
df.HourlyWindDirection.value_counts()

South       17608
Calm        11535
Variable     5570
West         4684
North        4396
East         3631
Name: HourlyWindDirection, dtype: int64

**Fix Wind Speed**

In [8]:
df.HourlyWindSpeed = df.HourlyWindSpeed.apply(lambda x: PrecipFix(x))

In [9]:
df.HourlyWindSpeed.value_counts()

0.0     11535
3.0      6350
5.0      6168
6.0      5662
7.0      4938
8.0      3905
9.0      2951
10.0     1887
11.0     1446
13.0      894
14.0      563
15.0      398
16.0      269
17.0      177
18.0      104
20.0       59
21.0       42
22.0       32
23.0       12
25.0       10
33.0        7
24.0        5
28.0        4
26.0        2
29.0        2
30.0        1
31.0        1
Name: HourlyWindSpeed, dtype: int64

In [10]:
with open("seattle_data.pkl", 'wb') as picklefile:
    pickle.dump(df, picklefile)

In [12]:
df.isna().sum()

DATE                        0
HourlyDryBulbTemperature    0
HourlyPrecipitation         0
HourlyRelativeHumidity      0
HourlySkyConditions         0
HourlyStationPressure       0
HourlyVisibility            0
HourlyWindDirection         0
HourlyWindSpeed             0
SHORTDATE                   0
DEMAND                      0
dtype: int64

In [14]:
df.rename(columns={'HourlyDryBulbTemperature':'Temperature', 'HourlyPrecipitation':'Precipitation', 'HourlyRelativeHumidity':'Humidity', 'HourlySkyConditions':'Condition', 'HourlyStationPressure':'Pressure', 'HourlyVisibility':'Visibility', 'HourlyWindDirection':'WindDirection', 'HourlyWindSpeed':'WindSpeed'}, inplace=True)

In [15]:
df.head()

,DATE,Temperature,Precipitation,Humidity,Condition,Pressure,Visibility,WindDirection,WindSpeed,SHORTDATE,DEMAND
0,2015-07-02 00:53:00,74.0,0.0,48.0,CLR,29.94,10.0,Calm,0.0,2015-07-02,1282.0
1,2015-07-02 01:53:00,72.0,0.0,53.0,CLR,29.94,10.0,Variable,3.0,2015-07-02,1277.0
2,2015-07-02 02:53:00,70.0,0.0,55.0,CLR,29.95,10.0,North,3.0,2015-07-02,1226.0
3,2015-07-02 03:53:00,68.0,0.0,59.0,CLR,29.96,10.0,Calm,0.0,2015-07-02,1186.0
4,2015-07-02 04:53:00,68.0,0.0,61.0,CLR,29.97,10.0,West,3.0,2015-07-02,1157.0


In [16]:
df.columns

Index(['DATE', 'Temperature', 'Precipitation', 'Humidity', 'Condition',
       'Pressure', 'Visibility', 'WindDirection', 'WindSpeed', 'SHORTDATE',
       'DEMAND'],
      dtype='object')

In [17]:
cols = ['DATE','SHORTDATE', 'Temperature', 'Precipitation', 'Humidity', 'Condition',
       'Pressure', 'Visibility', 'WindDirection', 'WindSpeed',
       'DEMAND']
df = df[cols]


In [18]:
df.head()

,DATE,SHORTDATE,Temperature,Precipitation,Humidity,Condition,Pressure,Visibility,WindDirection,WindSpeed,DEMAND
0,2015-07-02 00:53:00,2015-07-02,74.0,0.0,48.0,CLR,29.94,10.0,Calm,0.0,1282.0
1,2015-07-02 01:53:00,2015-07-02,72.0,0.0,53.0,CLR,29.94,10.0,Variable,3.0,1277.0
2,2015-07-02 02:53:00,2015-07-02,70.0,0.0,55.0,CLR,29.95,10.0,North,3.0,1226.0
3,2015-07-02 03:53:00,2015-07-02,68.0,0.0,59.0,CLR,29.96,10.0,Calm,0.0,1186.0
4,2015-07-02 04:53:00,2015-07-02,68.0,0.0,61.0,CLR,29.97,10.0,West,3.0,1157.0


In [19]:
with open("seattle_data.pkl", 'wb') as picklefile:
    pickle.dump(df, picklefile)